해당 파일은 Label-smoothing의 효과와 Hyperparameter Evolution 기법을 통한 Hyperparameter Tuning의 효과를 실험해본 결과를 담고 있다. <br>
아래 명령을 수행하면 최적의 하이퍼파라미터가 ./runs/evolve/{--name}/hyp_evolve.yaml 파일에 저장된다.

* python train.py --img 1024 --batch 16 --epochs 10 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './data/hyps/hyp.scratch-high.yaml' --label-smoothing 0.1 --name YoloV5x_Hyp_tuning --cache --evolve

[참고] 학습이 너무 오래 걸려서 16번 후 종료했다. (약 10~11시간 소요)

<h3> <strong> 1. 실험 </strong> </h3>

<h4> <strong>[비교] (기본 모델/label-smoothing 적용/label-smoothing 적용+Hyperparmeter Evolution)</strong> </h4>

**학습 명령**
* python train.py --img 1024 --batch 8 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './data/hyps/hyp.scratch-high.yaml' --name YoloV5x_base --cache
* python train.py --img 1024 --batch 8 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './data/hyps/hyp.scratch-high.yaml' --label-smoothing 0.1 --name YoloV5x_label_smoothing --cache
* python train.py --img 1024 --batch 8 --epochs 70 --data './data/dataset.yaml' --cfg './models/yolov5x.yaml' --weights yolov5x.pt --hyp './runs/evolve/YoloV5x_Hyp_tuning/hyp_evolve.yaml' --label-smoothing 0.1 --name YoloV5x_label_smoothing_Hyp_tuning --cache

**테스트 명령**
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_base/weights/best.pt' --save-txt --save-conf --name YoloV5x_base
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_label_smoothing/weights/best.pt' --save-txt --save-conf --name YoloV5x_label_smoothing
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_label_smoothing_Hyp_tuning/weights/best.pt' --save-txt --save-conf --name YoloV5x_label_smoothing_Hyp_tuning
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_label_smoothing_Hyp_tuning/weights/best.pt' --save-txt --save-conf --name YoloV5x_label_smoothing_Hyp_tuning_TTA --augment
* python detect.py --img 1024 --source './data/test' --weights './runs/train/YoloV5x_label_smoothing_Hyp_tuning/weights/best.pt' --save-txt --save-conf --name YoloV5x_label_smoothing_Hyp_tuning_IOU --augment --iou-thres 0.3

**[Exception ignored in: <function StorageWeakRef.__del__ at ...> Error]** 

해당 명령을 터미널에 입력하면 해결 가능하다고 한다.

* pip install --upgrade --force-reinstall torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0

아래는 제출을 위해 submission.csv 파일을 제작하기 위한 코드이다.

In [1]:
import pandas as pd
import os

df = pd.DataFrame(index=[i for i in range(4871)], columns=['PredictionString', 'image_id'])

yolo_path = './../yolov5/runs/detect/YoloV5x_label_smoothing_Hyp_tuning_IOU/labels'
for img_index in range(4871):
    image_id = str(img_index)
    while len(image_id)<4: image_id = '0'+image_id

    PredictionString = ''
    try: 
        txt_file = open(os.path.join(yolo_path, image_id+'.txt'), 'r')
        lines = txt_file.readlines()
    
        for line in lines:
            category, yolo_x, yolo_y, yolo_w, yolo_h, pred = (line.split())
            x_min = (float(yolo_x)-float(yolo_w)/2)*1024
            y_min = (float(yolo_y)-float(yolo_h)/2)*1024
            x_max = x_min+float(yolo_w)*1024
            y_max = y_min+float(yolo_h)*1024
            PredictionString += f'{category} {pred} {x_min} {y_min} {x_max} {y_max} '
        txt_file.close()
    except FileNotFoundError:
        pass
    
    df.iloc[img_index] = [PredictionString, 'test/'+image_id+'.jpg']

df.to_csv('./YoloV5x_label_smoothing_Hyp_tuning_IOU.csv', index=False)


<h3> <strong> 2. 결과 </strong> </h3>

(1) Base Model: 0.4348 / 아무것도 적용하지 않은 기본 base 모델임. <br>
(2) Label Smoothing: 0.4308 / Base Model에 label smoothing 적용시 valid loss 그래프는 더 안정적이지만 mAP 성능은 낮아짐. <br>
(3) LS + Hyperparameter Tuning: 0.4623 / hyper evolution 기법을 사용해 최적의 hyperparameter 조합을 찾으면 성능이 향상됨. <br>
(4) LS + HT + TTA: 0.5017. TTA 적용시 성능이 향상됨. <br>
(5) LS + HT + TTA + Reduced IOU: 0.4944 / iou-thres 수치 감소시 성능이 0.01 정도 하락됨. 단, multi-scale 모델의 경우 성능이 오름.